### Cyper-physical Systems Programming Exam
by Benedikte Petersen, Natalia Anna Prokopiuk, Stijn Kolkman

## About the project

The objective of the project is to adaption an orignial code changing video frames in to Dynamic vison sensor (DVS) frames, so it is able to do batch processing.

The original code is made by the Swiss National Competence Center for Robotics (https://github.com/SensorsINI/v2e). 

The main objective is to create a simulator that, instead of taking as input a frame of size W, H for each timestep, have to take torch tensor (already in GPU) with size B (batch), W, H.

## About the process

To convert the code to support batch processing 3 files had to be rewritten Newmain.py, Newemulator.py and emulator_utiliesNew.py. 

# About NewMain.py

The file is structured
- Configuration
    Run on cpu
- Read video files
    defineing path and batch size
- Define emulator settings and initialization
    Defining values such as threshold for detecting positie and negative events and minimum time between consecutive events at a pixel, preventing oversampling. and so on.
- Video Analyzing
    loop over the video's in the input folder to get the frames and the information (fps, num_of_frames, duration, delta_t, current_time) 
- Batch Video Frame Processing and Event Generation Loop
    This loop processes video frames from multiple video streams in parallel and performs initial pre-processing, such as converting frames to tensors and preparing them for subsequent computations.

    Convert numpy array into pytorch tensor

    Make a frame tensor storing height and width for each video

    Then create luma frame tensor by multiply the frame tensor with the weight. WHY

    The luma frame is then passed to the generate event function in emulator new. The function simulates events based on the frame

    Log event statistics for the batch

-Relaease resources

# about emulatorNew

The newMain call the generate_event function in emulator. The function take the luma frame and a time stamp and return the events

- Time mangement
    make sure the time is forward going and calculate a delta time
- Check if float 32
 

## About the code and user guide


The code in this repository is an adaption of the original code by the Swiss National Competence Center for Robotics (https://github.com/SensorsINI/v2e). 
The main change in this code is:

Batch Processing:
- Added functionality to process multiple video files simultaneously, leveraging GPU parallelism for improved efficiency.
- Supports batch-wise data loading and event generation.

To run the code follow the steps below which are copied from the original V2E github. Please not that you should clone this repository and not the original one.


## Local Installation (NEEDS TO BE EDITH)
There are 3 general steps
1. Make a _conda_ environment
2. Install _pytoch_ and other _conda_ distributed packages to this environment
3. Install the rest of the packages and _v2e_ to the conda enviroment with _pip_.

_pip_ is needed because some packages are not availble from the conda repositories. It is important to go in this order because _conda_ in general is not aware of _pip_ installs.

### Make conda environment
Install _v2e_ to a separate Python environment
such as `conda` environment:

```bash
conda create -n v2e python=3.10  # create a new environment
conda activate v2e  # activate the environment
```

### Use pip to install rest of packages and v2e (NEEDS TO BE EDITH)
After installing pytorch to your CUDA environment, to install v2e in developer mode (so your edits to source take effect immediately), run the following command in your terminal inside the activated conda environment. The `python -m pip install -e .` command runs _setup.py_ which installs more packages and adds a script to run _v2e_ from the command line to the _conda_ enviroment python path; see https://stackoverflow.com/questions/39023758/what-does-pip-install-dot-mean:
```bash
conda activate v2e # activate your env with pytoch already installed by conda
git clone https://github.com/SensorsINI/v2e
cd v2e
python -m pip install -e . # use pip to install requirements from setup.py in user mode (so your edits to source files in v2e take effect immediately
```

+ If you want an additional Windows GUI interface, you will need to install [Gooey](https://github.com/chriskiehl/Gooey) package. This package works the best on Windows:
    ```bash
    pip install Gooey
    ```
    On Linux, `Gooey` can be hard to install.

    For a sample of conversion using the gooey GUI, see https://youtu.be/THJqRC_q2kY

### Download SuperSloMo model (NEEDS TO BE EDITH)

We use the excellent [Super SloMo](https://people.cs.umass.edu/~hzjiang/projects/superslomo/) framework to interpolate the APS frames.
However, since APS frames only record light intensity, we  retrained it on grayscale images.

Download our pre-trained model checkpoint from Google Drive
[SuperSloMo39.ckpt](https://drive.google.com/file/d/1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx/view?usp=sharing) (151 MB) and save it to the _input_ folder. 
The default value of --slomo_model argument is set to this location.

Special thanks to Zhe He for recovering the checkpoint file.

### Download sample input data (NEEDS TO BE EDITH)
The sample input videos to try _v2e_ with are in [v2e-sample-input-data](https://drive.google.com/drive/folders/1oWxTB9sMPp6UylAdxg5O1ko_GaIXz7wo?usp=sharing) on google drive.

Download the [tennis.mov](https://drive.google.com/file/d/1dNUXJGlpEM51UVYH4-ZInN9pf0bHGgT_/view?usp=sharing)
video and put in the _input_ folder
to run the example below.

## Usage (NEEDS TO BE EDITH)

_v2e_ serves multiple purposes. Please read to code if you would like to adapt it for your own application. Here, we only introduce the usage for generating DVS events from conventional video and from specific datasets.

 


 




In [ ]:
"""
----------------------------------------------------------------------------
Project Name       : Full GPU DVS Simulator
File Name          : NewMain.py
Authors            : Benedikte Petersen, Natalia Anna Prokopiuk, Stijn Kolkman
Date Created       : 2024-12-04
Last Modified      : YYYY-MM-DD
Description        : This script simulates events from video frames using a
                     GPU-based event emulator. It processes video input to 
                     generate events suitable for neuromorphic vision processing.

Usage              : Run the script in the v2e conda environment. For details on
                     creating the environment, see:
                     https://github.com/StijnKolkman/Project_cpsp-v2eV3.git

Command            : python NewMain.py
----------------------------------------------------------------------------
"""
from v2ecore.emulatorNew import EventEmulator                       # Import the v2e simulator
import torch                                                        # Torch
import cv2                                                          # To read video using OpenCV
import glob

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #  This allows it to work on a cpu
video_folder = 'input/*.mov'

# Read video files
video_files = glob.glob(video_folder)                           #List of paths to the videos
batch_size = len(video_files)                                   #The batch size is equal to the amount of videos
if batch_size == 0:
    print("No video files found in the specified folder.")
    exit()

# define a emulator (set the settings of the emulator)
emulatorNew = EventEmulator(
    pos_thres           = 1,
    neg_thres           = 1,
    sigma_thres         = 0.03,
    cutoff_hz           = 200,
    leak_rate_hz        = 1, 
    batch_size          = batch_size,
    device              = device,
    refractory_period_s = 0.01,
    output_folder       = 'output/',

    # having this will allow H5 output
    dvs_h5              = 'eventsH5',

    # Other possible output format
    dvs_aedat2          = None, #currently doesnt work with batch processing
    dvs_aedat4          = None, #currently doesnt work with batch processing
    dvs_text            = None, #currently doesnt work with batch processing

    # True when the input is in hdr format (then it will skip the linlog conversion step)
    hdr                 = False,

    # If True, photoreceptor noise is added to the simulation
    photoreceptor_noise = False,

    #CSDVS --> This will enable csdvs, this function currently does not work with batch processing, so keep at None
    cs_lambda_pixels    = None,

    #scidvs
    scidvs              = False,

    # Parameter to show an image and to save it to an avi file. If show_dvs_model_state is enabled it will output the frames and will wait for a key press before it continues. Pressing 'x' will quite the process
    show_dvs_model_state = ['diff_frame','lp_log_frame', 'log_new_frame'], # options:['all','new_frame', 'log_new_frame','lp_log_frame', 'scidvs_highpass', 'photoreceptor_noise_arr', 'cs_surround_frame','c_minus_s_frame', 'base_log_frame', 'diff_frame'])
    output_height       =200, #output height of the windowg
    output_width        =200,
    save_dvs_model_state= True,

    #define shot noise or not 
    shot_noise_rate_hz  = 0,
    label_signal_noise  = False, #Currently doesnt work with batch processing and also doesnt work in the original code

    #record the state of a single pixel (input is tuple)
    record_single_pixel_states = (1, 10, 20) #example tuple containing pixel info: batch, height, width
)
# **IMPORTANT** make torch static, likely get faster emulation (might also cause memory issue)
torch.set_grad_enabled(False)

# Initialize resources and tensors
caps        = []                                                                    #here the videos will be saved
fps           = 0                                                                   #fps of the videos, should be same for th videos
num_of_frames = 0                                                                   #total number of frames. Should be the same for every video
duration      = 0                                                                   # Duration of the videos, should be the same for every video
delta_t       = 0                                                                   #The time between two frames. IS ASSUMED TO BE THE SAME FOR EVERY VIDEO
current_time  = 0                                                                   #Current time is not a tensor anymore since we can assume that every video has the same size

#loop over the video's in the input folder to get the frames and the information (fps, num_of_frames, duration, delta_t, current_time)
#maybe add something here to check that the videos really have the same fps and delta_t
print() 
for i, video_file in enumerate(video_files): 
    print(f"Opening video {i+1}: {video_file}")
    cap = cv2.VideoCapture(video_file)

    if not cap.isOpened(): 
        print(f"Error opening video file: {video_file}")
        continue

    # Append the VideoCapture object to the list
    caps.append(cap)

    # Get the information
    fps = cap.get(cv2.CAP_PROP_FPS)
    print("FPS: {}".format(fps))
    num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Num of frames: {}".format(num_of_frames))
    duration = num_of_frames/fps
    print("Clip Duration: {}s".format(duration))
    delta_t = 1/fps                                    
    print("Delta Frame Time: {}s".format(delta_t))
    print() 

new_events = None                                           #Initialise the new_events. Will be filled by the emulator with events
idx        = 0                                              #Initialise counter
N_frames   = 100                                              #Only Emulate the first N_frames of every video TODO: LATER REMOVE JUST TO MAKE TESTING TAKE LESS TIME!!!
ret        = torch.zeros(batch_size,device=device)          #Tensor that stores the return value of cap.read()


max_height  = 720                                                                        # Example max height
max_width   = 1280                                                                       # Example max width
channels    = 3                                                                          # RGB
frame_tensor = torch.zeros((batch_size, max_height, max_width, channels), device=device) # Tensor containing the frames
luma_frame_tensor = torch.zeros((batch_size, max_height, max_width), device=device)      # Tensor containing the luma_frames
weights     = torch.tensor([0.299, 0.587, 0.114],device=device).view(1, 1, 1, 3)         # Weights for transfer to grayscale, see:https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html#color_convert_rgb_gray 


while True:
    all_ret_false = True  # Flag to determine if all videos are done

    for i, cap in enumerate(caps):
        ret, frame_read = cap.read() 
        if ret:
            all_ret_false = False
            frame_tensor[i] = torch.from_numpy(frame_read).float().to(device)  # Convert frame (np array) to tensor
            #print("Size of frame_tensor[{}]: {}".format(i, frame_tensor[i].size()))
        else:
            # Append a zero tensor if video is finished
            frame_tensor[i] = torch.zeros((max_height, max_width), device=device) #Not really needed anymore since it is assumed that all files will have the same number of frames

    if all_ret_false:  # Stop if all videos are done
        break
    #print("Size of frame_tensor: {}".format(frame_tensor.size()))
    luma_frame_tensor = (frame_tensor * weights).sum(dim=-1)  # Compute luma frame
    #print("Size of luma_frame_tensor: {}".format(luma_frame_tensor.size()))
    # Generate events
    print("="*50)
    print(f"Processing frame_batch {idx + 1} of in total {N_frames} frame_batches")
    print("="*50)
    new_events = emulatorNew.generate_events(luma_frame_tensor, current_time)

    # Update current time
    current_time += delta_t

    # printing some events to look at the output
    if current_time > delta_t:
        print('Now going to print some events for you (batch, time, x, y, polarity):')
        print(new_events[0:4,:])

    # Log event statistics for the batch
    if new_events is not None:
        num_events = new_events.shape[0]
        start_t = new_events[0, 1]
        end_t = new_events[-1, 1]
        event_time = (new_events[-1, 1]-new_events[0, 1])
        event_rate_kevs = (num_events/delta_t)/1e3
        print("Number of Events: {}\n"
            "Duration: {}s\n"
            "Start T: {:.5f}s\n"
            "End T: {:.5f}s\n"
            "Event Rate: {:.2f}KEV/s".format(
                num_events, event_time, start_t, end_t,
                event_rate_kevs))

    print("="*50)
    idx += 1
    if idx >= N_frames:  # Limit for testing
        break

# Release resources
for cap in caps:
    cap.release()
print("Processing complete. Resources released.")


Opening video 1: input\tennis.mov
FPS: 59.94007280921804
Num of frames: 1551
Clip Duration: 25.875844444444443s
Delta Frame Time: 0.016683329751414858s

Opening video 2: input\tennis2 - Copy (2).mov
FPS: 59.94007280921804
Num of frames: 1551
Clip Duration: 25.875844444444443s
Delta Frame Time: 0.016683329751414858s

Processing frame_batch 1 of in total 100 frame_batches


IIR lowpass filter update has large maximum update eps=20.96 from delta_time/tau=0.0167/0.000796


Processing frame_batch 2 of in total 100 frame_batches


IIR lowpass filter update has large maximum update eps=20.96 from delta_time/tau=0.0167/0.000796


Maximum number of events generated in each batch: [3, 3]
The size of ts is:  torch.Size([2, 3])
Now going to print some events for you (batch, time, x, y, polarity):
[[ 0.0000000e+00  5.5611096e-03  4.4900000e+02  2.1700000e+02
  -1.0000000e+00]
 [ 1.0000000e+00  5.5611096e-03  1.1640000e+03  4.8700000e+02
   1.0000000e+00]
 [ 1.0000000e+00  5.5611096e-03  9.9800000e+02  2.6800000e+02
  -1.0000000e+00]
 [ 1.0000000e+00  5.5611096e-03  1.0100000e+03  4.4200000e+02
  -1.0000000e+00]]
Number of Events: 16619
Duration: 0.011122219264507294s
Start T: 0.00556s
End T: 0.01668s
Event Rate: 996.14KEV/s
Processing frame_batch 3 of in total 100 frame_batches


...

Maximum number of events generated in each batch: [3, 3]
The size of ts is:  torch.Size([2, 3])
Now going to print some events for you (batch, time, x, y, polarity):
[[ 0.00000000e+00  1.05661094e-01  1.12700000e+03  1.32000000e+02
  -1.00000000e+00]
 [ 0.00000000e+00  1.05661094e-01  1.07500000e+03  2.06000000e+02
   1.00000000e+00]
 [ 1.00000000e+00  1.05661094e-01  8.61000000e+02  3.30000000e+02
  -1.00000000e+00]
 [ 0.00000000e+00  1.05661094e-01  9.51000000e+02  1.05000000e+02
   1.00000000e+00]]
Number of Events: 41184
Duration: 0.011122211813926697s
Start T: 0.10566s
End T: 0.11678s
Event Rate: 2468.57KEV/s
Processing frame_batch 9 of in total 100 frame_batches


IIR lowpass filter update has large maximum update eps=20.90 from delta_time/tau=0.0167/0.000796


Maximum number of events generated in each batch: [3, 3]
The size of ts is:  torch.Size([2, 3])
Now going to print some events for you (batch, time, x, y, polarity):
[[ 0.0000000e+00  1.2234442e-01  1.1170000e+03  3.4500000e+02
   1.0000000e+00]
 [ 1.0000000e+00  1.2234442e-01  1.0880000e+03  2.3300000e+02
   1.0000000e+00]
 [ 0.0000000e+00  1.2234442e-01  9.5700000e+02  2.2400000e+02
   1.0000000e+00]
 [ 1.0000000e+00  1.2234442e-01  9.6200000e+02  4.2000000e+02
  -1.0000000e+00]]
Number of Events: 47084
Duration: 0.011122211813926697s
Start T: 0.12234s
End T: 0.13347s
Event Rate: 2822.22KEV/s
Processing frame_batch 10 of in total 100 frame_batches


IIR lowpass filter update has large maximum update eps=20.96 from delta_time/tau=0.0167/0.000796
Supressing further warnings about inaccurate IIR lowpass filtering; check timestamp resolution and DVS photoreceptor cutoff frequency


Maximum number of events generated in each batch: [3, 3]
The size of ts is:  torch.Size([2, 3])
Now going to print some events for you (batch, time, x, y, polarity):
[[ 0.0000000e+00  1.3902774e-01  1.0960000e+03  3.9200000e+02
   1.0000000e+00]
 [ 1.0000000e+00  1.3902774e-01  9.6700000e+02  3.5400000e+02
  -1.0000000e+00]
 [ 0.0000000e+00  1.3902774e-01  9.0200000e+02  2.3200000e+02
  -1.0000000e+00]
 [ 0.0000000e+00  1.3902774e-01  9.8000000e+02  3.5400000e+02
  -1.0000000e+00]]
Number of Events: 41596
Duration: 0.01112222671508789s
Start T: 0.13903s
End T: 0.15015s
Event Rate: 2493.27KEV/s
Processing frame_batch 11 of in total 100 frame_batches
Maximum number of events generated in each batch: [3, 3]
The size of ts is:  torch.Size([2, 3])
Now going to print some events for you (batch, time, x, y, polarity):
[[ 0.0000000e+00  1.5571108e-01  1.0770000e+03  2.1800000e+02
   1.0000000e+00]
 [ 1.0000000e+00  1.5571108e-01  1.0580000e+03  2.3600000e+02
   1.0000000e+00]
 [ 1.0000000e+00 

In [3]:

from v2ecore.renderer import EventRenderer
import h5py
import numpy as np  # Ensure numpy is imported for handling arrays


file_path = 'output/eventsH5.h5' # file path for the h5 file containing the events


renderer = EventRenderer(
    full_scale_count=1,
    output_path="output",
    dvs_vid="dvs_video.avi",
    preview=True,
    #exposure_mode=ExposureMode.DURATION,
    exposure_value=1/30.0,  # 30 FPS
    avi_frame_rate=30,
)

with h5py.File(file_path, 'r') as hdf:
    dataset_name = 'events'
    if dataset_name in hdf:
        dataset = hdf[dataset_name]
        print(f"\nShape of the dataset '{dataset_name}': {dataset.shape}")
        print(f"Data type of the dataset: {dataset.dtype}")

        # Load the entire dataset into memory
        all_data = dataset[:]  # This loads all the data into a NumPy array

        # Convert the relevant columns to float if necessary
        all_data = all_data.astype(np.float32)  # Convert entire array to float32

        all_data[:, 1] = all_data[:, 1] / 1e6 #change time to seconds
        print(f"Loaded {len(all_data)} entries from '{dataset_name}'.")

        # Now you can process all_data as needed
        # For example, printing the first few rows:
        print(all_data[:10])  # Print the first 10 entries to verify

    else:
        print(f"Dataset '{dataset_name}' not found in the file.")

#test = all_data[1][1]
#print(test)

# Filter rows where the first column is equal to 0
filtered_data = all_data[all_data[:, 0] == 0] # 0 means that you are gonna render the first video. 
# Divide the second column by 1000
filtered_data_no_first_col = filtered_data[:, 1:]  # Select all columns except the first
height, width = 720, 1280
renderer.render_events_to_frames(filtered_data_no_first_col, height, width)


'''
events = np.array([
    [0.01, 10, 20, 1],
    [0.02, 15, 25, -1],  ...
])  
# Events with [timestamp, x, y, polarity]
height, width = 128, 128
renderer.render_events_to_frames(events, height, width)
'''



Shape of the dataset 'events': (4494801, 5)
Data type of the dataset: uint32
Loaded 4494801 entries from 'events'.
[[0.000e+00 5.561e-03 4.490e+02 2.170e+02 0.000e+00]
 [1.000e+00 5.561e-03 1.164e+03 4.870e+02 1.000e+00]
 [1.000e+00 5.561e-03 9.980e+02 2.680e+02 0.000e+00]
 [1.000e+00 5.561e-03 1.010e+03 4.420e+02 0.000e+00]
 [0.000e+00 5.561e-03 1.018e+03 4.290e+02 0.000e+00]
 [1.000e+00 5.561e-03 1.051e+03 3.620e+02 0.000e+00]
 [1.000e+00 5.561e-03 1.021e+03 1.080e+02 0.000e+00]
 [1.000e+00 5.561e-03 1.012e+03 3.020e+02 0.000e+00]
 [0.000e+00 5.561e-03 1.110e+03 7.900e+01 0.000e+00]
 [0.000e+00 5.561e-03 1.164e+03 4.750e+02 1.000e+00]]


'\nevents = np.array([\n    [0.01, 10, 20, 1],\n    [0.02, 15, 25, -1],  ...\n])  \n# Events with [timestamp, x, y, polarity]\nheight, width = 128, 128\nrenderer.render_events_to_frames(events, height, width)\n'